In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import utils
import torchvision
from torchvision.io import image
from torchvision.transforms.functional import to_pil_image
import pandas as pd
import time
import numpy as np
import collections
import re
import random
import math

In [2]:
batch_size, num_steps = 32, 35
# train_iter, vocab = utils.load_data_txt(utils.sanguo_txt_path, batch_size, num_steps)
train_iter, vocab = utils.load_data_txt(utils.santi_txt_path, batch_size, num_steps)
print(len(vocab))

1464


In [3]:
# 独热编码
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

# 每次采样的小批量数据形状是二维张量： （批量大小，时间步数）。 one_hot函数将这样一个小批量数据转换成三维张量， 张量的最后一个维度等于词表大小（len(vocab)）。 我们经常转换输入的维度，以便获得形状为 （时间步数，批量大小，词表大小）的输出。 这将使我们能够更方便地通过最外层的维度， 一步一步地更新小批量数据的隐状态。

In [4]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, len(vocab)).shape

torch.Size([5, 2, 1464])

In [5]:
# 初始化模型参数
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal(shape=(num_inputs, num_hiddens))
    W_hh = normal(shape=(num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal(shape=(num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 统一管理梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [6]:
# 初始化隐状态（即第一个h）
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [7]:
# rnn一个时间步内计算隐状态和输出
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)    # tanh定义域（-inf，inf），值域（-1,1）
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    # 在第一个维度拼接，cat后shape: (时间步数量 * 批量大小, 词表大小)，同时返回最后一个隐状态
    return torch.cat(outputs, dim=0), (H,)

In [8]:
class RNNModelScratch:
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device, get_params_fn, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params_fn(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
        self.device = device

    def __call__(self, X, state):
        X = X.to(self.device)
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, self.device)

In [9]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, utils.try_gpu(), get_params, init_rnn_state, rnn)
first_state = net.begin_state(X.shape[0], device=utils.try_gpu())
Y, new_state = net(X, first_state)
# shape:(时间步数量 * 批量大小, 词表大小), shape:(批量大小，隐藏单元数)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 1464]), 1, torch.Size([2, 512]))

In [10]:
# 预测

def predict_ch8(prefix, num_preds, net, vocab, device):
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        # 只更新state（隐状态），不记录输出
        _, state = net(get_input(), state)
        outputs.append(vocab[y])    # output只收集实际值
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [11]:
predict_ch8('cao cao ', 10, net, vocab, utils.try_gpu())

'cao cao 悟阅波换膀大1银平主'

In [12]:
# 梯度裁剪

def grad_clipping(net, theta):
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [13]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    time_0 = time.time()
    state = None
    # 训练损失之和,词元数量
    loss_sum = 0
    token_sum = 0
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        loss_sum += l * y.numel()
        token_sum += y.numel()
    return math.exp(loss_sum / token_sum), token_sum / (time.time() - time_0)

In [14]:
def train_ch8(net, train_iter, vocab, lr, num_epochs, device, predict_prefix, use_random_iter=False, predict_len=200):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: utils.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, predict_len, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 50 == 0:
            print(predict(predict_prefix[0]))
            print(f'epoch: {epoch+1}/{num_epochs}, ppl: {ppl}')
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict(predict_prefix[0]))
    print(predict(predict_prefix[1]))

In [15]:
# predict_prefix = ['cao cao', 'kingdom']
predict_prefix = ['三体组织', '物理学']

In [16]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, utils.try_gpu(), predict_prefix=predict_prefix)

三体组织的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 50/500, ppl: 454.7551229676679
三体组织，但是在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有，他们在有
epoch: 100/500, ppl: 324.00697930012973
三体组织的，因是的一个，我们在一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，我们是一个，
epoch: 150/500, ppl: 159.47543172991965
三体组织，我们是一个步动的电现。”“我们的一种，但我们是这个，他们的物理理论的大芒。作“我们的信非，他我们了这个，他们是我们个我的，他们还时天，更观将到一个公认，它是这个计算机开，他这个个度，这次是一个个现的，我们是一个，但且出到一个公认，它是这个计算机开，他这个个度，这次是一个个现的，我们是一个，但且出到一个公认，它是这个计算机开，他这个个度，这次是一个个现的，我们是一个，但且出到一个公认，它是这个计算
epoch: 200/500, ppl: 48.52726740480826
三体组织星中，在后于，这个级别得了，‘是从子计

In [17]:
# 随机抽样方法的结果
net = RNNModelScratch(len(vocab), num_hiddens, utils.try_gpu(), get_params, init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, utils.try_gpu(), use_random_iter=True, predict_prefix=predict_prefix)

三体组织，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这
epoch: 50/500, ppl: 456.97646091083055
三体组织的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的一个，但是的，他的，我们的
epoch: 100/500, ppl: 324.6506408298819
三体组织的，因是这个科的，我们是一个的飞的，他们在这个的飞究，但是一个计的，它是一个科的，我们是一个的学的，他们在这个的飞究，但是一个计的，它是一个科的，我们是一个的学的，他们在这个的飞究，但是一个计的，它是一个科的，我们是一个的学的，他们在这个的飞究，但是一个计的，它是一个科的，我们是一个的学的，他们在这个的飞究，但是一个计的，它是一个科的，我们是一个的学的，他们在这个的飞究，但是一个计的，它是一个科的
epoch: 150/500, ppl: 170.02002650161987
三体组织子的问量。”“文明的景句。这种，我们能自己的飞船是，也的，作次的创响，只究出的螺旋，认，它的技术，他们的任理理论，不管的金度。，她文体的景星的，你们生的理星点术，它管的一个点枪，而管的金属。，她文明的景星的，你们生的理星点术，它管的一个点枪，而管的金属。，她文明的景星的，你们生的理星点术，它管的一个点枪，而管的金属。，她文明的景星的，你们生的理星点术，它管的一个点枪，而管的金属。，她文明的景星的，
epoch: 200/500, ppl: 79.70863425967389
三体组织，我是这个计算机来的，他们的创响是父么

# 循环神经网络的简洁实现

In [18]:
batch_size, num_steps = 32, 35
# train_iter, vocab = utils.load_data_txt(utils.sanguo_txt_path, batch_size, num_steps)
train_iter, vocab = utils.load_data_txt(utils.santi_txt_path, batch_size, num_steps)
print(len(vocab))

1464


In [19]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

In [20]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 32, 256])

In [21]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [22]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device=device),
                    torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device=device))

In [23]:
device = utils.try_gpu()
net = RNNModel(rnn_layer, vocab_size=len(vocab))
net = net.to(device)

In [24]:
predict_prefix = ['三体组织', '物理学']
predict_ch8(predict_prefix[0], 10, net, vocab, device)

'三体组织墨求喊墨种疾任墨近求'

In [25]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, device, predict_prefix=predict_prefix)

三体组织的的的的的的的的的，他，，的，的，你的，他，，，的，这的，他的的的的的的的的的的，他，，的，的，的，这的，他，的，的，这的，他，的，”，这的，他，，，的的的的的的的的的的，他，，的，的，的，这的，他，的，的，这的，他，的，”，这的，他，，，的的的的的的的的的的，他，，的，的，的，这的，他，的，的，这的，他，的，”，这的，他，，，的的的的的的的的的的，他，，的，的，的，这的，他，的，的，这的，他，的
epoch: 50/500, ppl: 360.63281154463596
三体组织的人在，但我们是一个的数学，但是一个的，你是将了一个，我们是一个的基的，但有这个的数量，他们一个，我们是一个的技术，但是一个的大的，他是这个的数态，他们一个，我们是一个的技术，但是一个的大的，他是这个的数态，他们一个，我们是一个的技术，但是一个的大的，他是这个的数态，他们一个，我们是一个的技术，但是一个的大的，他是这个的数态，他们一个，我们是一个的技术，但是一个的大的，他是这个的数态，他们一个，我
epoch: 100/500, ppl: 149.11101922395127
三体组织的地质子，这次个一个，但上原到一个更子的一利，他至一个科学的数学。”“我们，你自己战择识一个的运的，只至出个大的基量，他们以一个人态。”“我是一个点数的大率。”“我们的信度官统的计算机，这些个被着，但新了一个种子的电机。”“我们的信度，他好出一个世星的荣础。”“我们的信孩官，我是这个军星的高人，但有了一个人号的荣础。”“我们的信度官，我是这个军星的高人，但有了一个人号的荣础。”“我们的信度官，我是
epoch: 150/500, ppl: 46.77790903146843
三体组织子地、焰夜地学、，在人类时到，这个用到，但需这个军大的，因上不由都了。”“这有，他们要自己识了那个。”“我的是一点.　红岸之一的，他与是目前的兴间，更是以两舰的态事和图多机需限知道。”，但是这个发别中的，示流的物理是，对论出了用行星的文础。但然这是的人能要的，我们这种们短生了层的，以是他们军星的主动。，我们无随过，因为了在这个然术的第成，他们不可以了，但文在的计算能，根经个时，你们所了，但管在目的
epoch: 200/500, ppl: 18.31529130589605
三体组织。”“我有要不需是，你是高速失算机来为